In [ ]:
import boto3

import pandas as pd
from io import StringIO, BytesIO
from datetime import datetime, timedelta

import sklearn
import matplotlib.pyplot as plt


In [ ]:
class Extract():
    def __init__(self, arg_date, src_format, src_bucket, trg_bucket, columns, key, s3, bucket):

        # Parameters/Configurations
        self.arg_date = arg_date
        self.src_format = src_format
        self.src_bucket = src_bucket
        self.trg_bucket = trg_bucket
        self.columns = columns
        self.key = key
        self.s3 = s3
        self.bucket = bucket
        
    def read_csv_to_df(self, filename,bucket):
        csv_obj = bucket.Object(key=filename).get().get('Body').read().decode('utf-8')
        data = StringIO(csv_obj)
        df = pd.read_csv(data, delimiter=',')
        return df

    def return_objects(self):
        arg_date_dt = datetime.strptime(self.arg_date, self.src_format).date() - timedelta(days=1)
        print(arg_date_dt)        
        objects = [obj for obj in self.bucket.objects.all() if datetime.strptime(obj.key.split('/')[0], self.src_format).date() >= arg_date_dt]
        return objects

    def extract(self, objects,columns,bucket):
        df_all = pd.concat([self.read_csv_to_df(obj.key, bucket) for obj in objects], ignore_index=True)
        df_all = df_all.loc[:, columns]
        df_all.dropna(inplace=True)
        print(df_all)
        return df_all


In [ ]:
class Transform():

    def transform_report(self, df_all, arg_date, columns):

        df_all = df_all.loc[:, columns]
        df_all.dropna(inplace=True)

        df_all = df_all.loc[(df_all["Time"] >= '08:00') & (df_all["Time"]<='12:00'),['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']]
        df_all.shape
        df_all = df_all.round(decimals=2)
        df_all = df_all[df_all.Date >= arg_date]

        ## Deviacion Estandar
        df_all['STD'] = df_all[["StartPrice", "EndPrice"]].std(axis = 1)
        df_all

        ## Conversion de EndPrice (Euros) a pesos Mexicanos

        df_all['MXN'] = df_all['EndPrice'] * 22.83
        df_all

        ##

        df_all.plot(x='EndPrice', y='MXN', style='o')

        plt.title('EndPrice - MXN')
        plt.xlabel('EndPrice')
        plt.ylabel('MXN')
        plt.show()

        x=df_all['EndPrice'].values.reshape(-1,1)
        y=df_all['MXN'].values.reshape(-1,1)

        from sklearn.model_selection import train_test_split
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=0)

        from sklearn.model_selection import LinearRegression
        lr = LinearRegression()
        lr.fit(x_train, y_train)

        print(lr.intercept_)
        print(lr.coef_)

        y_pred = lr.predict(x_test)

        df_prediccion = pd.DataFrame({'Actual': y_test.flatten(), 'Prediccion': y_pred.flatten()})
        df_prediccion

        plt.scatter(x_test, y_test, color='blue')
        plt.plot(x_test, y_pred, color='red', linewidth=2)
        plt.show()

        df_all=pd.concat([df_all, df_prediccion])
        df_all = df_all.infer_objects()
        df_all.dtypes
        df_all
        
        print(df_all)
        return df_all
        

In [ ]:
class Load():
        
    def write_df_to_s3(self, df_all,s3,trg_bucket,key):
        out_buffer = BytesIO()
        df_all.to_parquet(out_buffer, index=False)
        bucket_target = s3.Bucket(trg_bucket)
        bucket_target.put_object(Body=out_buffer.getvalue(), Key=key)
        return bucket_target

    def load(self, bucket_target):
        objKey=[]
        for obj in bucket_target.objects.all():
            objKey.append(obj.key)
        prq_obj = bucket_target.Object(key=objKey[-1]).get().get('Body').read()
        data = BytesIO(prq_obj)
        return data

    def etl_report(self, bucket_target):
        df_report = pd.read_parquet(self.load(bucket_target))
        return df_report

In [ ]:

def main():
    arg_date = '2022-04-07'
    src_format = '%Y-%m-%d'
    src_bucket = 'deutsche-boerse-xetra-pds'
    trg_bucket = 'xetra-bucket-gerardo-2022'
    columns = ['ISIN', 'Date', 'Time', 'StartPrice', 'MaxPrice', 'MinPrice', 'EndPrice', 'TradedVolume']
    key = 'xetra_daily_report_' + datetime.today().strftime("%Y%m%d_%H%M%S") + '.parquet'
    s3 = boto3.resource('s3')
    bucket = s3.Bucket(src_bucket)

    ext = Extract(arg_date, src_format, src_bucket, trg_bucket, columns, key, s3, bucket)
    
    objects = ext.return_objects()
    df_all = ext.extract(objects,columns,bucket)
    
    tran = Transform(df_all, arg_date, columns)
    df_all = tran.transform_report(df_all, arg_date, columns)
    

    ld = Load()
    bucket_target = ld.write_df_to_s3(df_all,ext.s3,ext.trg_bucket,ext.key)
    

    print(ld.etl_report(bucket_target))

In [ ]:
main()